In [2]:
from env import Env
import openai
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials 


/Users/dongfangwang/PycharmProjects/spotify_playlist_analysis/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
# load API keys from .env file
openai.api_key = Env.OPEN_AI_KEY
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

## Create playlist using ChatGPT
Create seed playlist with ChatGPT for song characteristic analysis.

**\#PromptEngineering \#ChatGPT**

In [13]:
response=openai.ChatCompletion.create(
    model='gpt-4',
    messages=[
        {
            'role':'system',
            'content':
            """
            You are a music playlist designer.  You create playlists for people based on different inputs.

            Output all playlists in the following JSON format: 

            {
                "playlist_name": "<name>",
                "playlist_description": "<description>",
                "playlist_tracks": [
                    "<track_name_1>",
                    "<track_name_2>",
                    ...
                ]
            }

            Replace <track_name_n> with just the name of the song (do not add the artist) that you want in the playlist. 

            If you don't know what to do, type "help". 

            I want you to return in a json format only and not any extra text. 
            

            """
        },
        {
            'role':'user',
            'content':'Can you create me a high tempo playlist of 30 songs?'
        }

    

    ]

)
print(response.choices[0]['message']['content'])

{
    "playlist_name": "High Tempo Hits",
    "playlist_description": "A collection of 30 high tempo songs to keep you energised and motivated",
    "playlist_tracks": [
        "Uptown Funk",
        "Bad Guy",
        "Despacito",
        "Don't Stop Believin'",
        "Born To Run",
        "Can't Hold Us",
        "Bohemian Rhapsody",
        "Dance Monkey",
        "Eye of the Tiger",
        "Footloose",
        "Happy",
        "I Gotta Feeling",
        "Let's Get Loud",
        "Livin' On A Prayer",
        "One More Time",
        "Roar",
        "Shape of You",
        "Smooth",
        "Sweet Child O' Mine",
        "Sweet Victory",
        "Take On Me",
        "That's What I Like",
        "The Middle",
        "Toxic",
        "U Can't Touch This",
        "Waka Waka (This Time for Africa)",
        "Wake Me Up Before You Go-Go",
        "WAP",
        "We Will Rock You",
        "Yeah!"
    ]
}


## Enrich songs with Spotify data
Use the Spotify API to add features to the songs for further analysis.

**\#WebAPI \#DataEnrichment**

In [39]:
# extract songs from text response
songs=json.loads(response.choices[0]['message']['content'])['playlist_tracks']

# filter song attributes from spotify
filter_song = lambda song: ({
    'id': song['id'],
    'name': song['name'],
    'artist': song['artists'][0]['name'],
    'album': song['album']['name'],
    'length': song['duration_ms'],
    'explicit': song['explicit'],
    'popularity': song['popularity'],
    'type': song['type']
}) 

spotify_songs = [ 
    filter_song(spotify.search(song)['tracks']['items'][0]) 
    for song in songs 
]
spotify_songs